In [1]:
import numpy as np

terrain_data = np.load(f'processed_terrain_data.npy')
terrain_labels = np.load(f'terrain_data_labels.npy')
terrain_columns = np.load('terrains_columns_metadata.npy')

# Clasificador SVM

In [2]:
import pickle
from pprint import pprint
from sklearn import svm
from sklearn.model_selection import cross_validate

CV = 8
clf = svm.SVC(kernel = 'linear', class_weight='balanced')
cv_results = cross_validate(clf, terrain_data, terrain_labels, cv=CV, scoring=('accuracy', 'recall_micro'))
print('SVM Radial Accuracy: ', cv_results['test_accuracy'].sum()/CV)
pprint(cv_results)

with open(f'terrain_svm_linear.pkl', 'wb') as f:
    pickle.dump(clf, f)

SVM Radial Accuracy:  0.6813397343308273
{'fit_time': array([412.20055151, 632.480901  , 485.57682395, 284.43923569,
       292.5271163 , 275.13119411, 387.55154991, 231.38961601]),
 'score_time': array([0.04476261, 0.04135418, 0.02492881, 0.02907443, 0.03001761,
       0.0285964 , 0.03046346, 0.02697563]),
 'test_accuracy': array([0.70719603, 0.78908189, 0.64019851, 0.7369727 , 0.64516129,
       0.57568238, 0.71712159, 0.63930348]),
 'test_recall_micro': array([0.70719603, 0.78908189, 0.64019851, 0.7369727 , 0.64516129,
       0.57568238, 0.71712159, 0.63930348])}
